### this script construct conterfactual research profile that countries only recieve domestic fund. 

Assign publications to author country using full counting first, and then for each country, following publications are retained:
- articles that are not funded
- articles that are exclusively funded by the country itself

The output of the script is the number of publications of countries in year and discipline

In [1]:
import pandas as pd

In [2]:
pubs_path="../../../data/nf_folder/Data/DerivedData/CleanedRawData/pub.pkl"

pub_filter_path="../../../data/nf_folder/Data/DerivedData/Derived/dep-FundInternational/pub_noInt_fund.csv"

In [3]:
# Parameters
pubs_path = "../data/NationalFunding/Data/simulated_data/Data/DerivedData/CleanedRawData/pub.pkl"
pub_filter_path = "../data/NationalFunding/Data/simulated_data/Data/DerivedData/Derived/dependence/pub_noint_fund.csv"


In [4]:
pub_df=pd.read_pickle(pubs_path)
pub_df.head()

,id,year,dis,type,author,funder,author_distinct,funder_distinct,funded,IntCol,cofund,intfund
0,0,2014,29,1,[Canada],"[United States, Brazil, United States, Republi...",[Canada],"[Republic of Serbia, China, Brazil, United Sta...",1,0,1,1
1,1,2017,32,1,"[Russia, Spain]","[Austria, China, China, China, United States, ...","[Russia, Spain]","[China, United States, Austria, Russia, Japan]",1,1,1,1
2,2,2012,15,1,[Turkey],Not-Funded,[Turkey],Not-Funded,0,0,0,0
3,3,2010,53,1,"[United States, Iran]",[China],"[Iran, United States]",[China],1,1,0,1
4,4,2015,22,1,[Denmark],Not-Funded,[Denmark],Not-Funded,0,0,0,0


In [5]:
author_df=pub_df[['id','year','dis','author_distinct','funder_distinct']]
author_df=author_df.explode('author_distinct')

# detect whether the paper is foreign funded or not

In [6]:
def assign_label(author,funders):
    label=1
    if funders=='Not-Funded':
        label=0
    elif len(funders)==1 and funders[0]==author:
        label=0
    return label
author_df['label']=author_df.apply(lambda x:assign_label(x.author_distinct,x.funder_distinct),axis=1)

In [7]:
author_df=author_df[author_df.label==0]
author_df=author_df.groupby(['author_distinct','year','dis']).size().reset_index()

In [8]:
author_df.columns=['cntry','year','dis','count']
author_df.to_csv(pub_filter_path,index=False)